# Router

A HTTP Request router for coil

In [1]:
@require "." @route router Request Response;

## API

### `@route(fn, router, path)`

This macro is purely here to save you having to come up with names for the functions which handle requests

In [2]:
@route(router, "user/:(\\d+)") do r::Request{:GET}, id::Int
  Response(200, "getting user #$id")
end

@route(router, "user/:(\\d+)") do r::Request{:PUT}, id::Int
  Response(200, "putting user #$id")
end;

The router can then be called with a `Request` object and it will dispatch to the correct handler based on the Uri path contained within the `Request`. The handler will then presumably return a HTTP `Response` but it's not enforced

In [3]:
router(Request(IOBuffer("GET /user/1")))

Response(200,Dict("Content-Length"=>"15"),"getting user #1")

In [4]:
router(Request(IOBuffer("PUT /user/1")))

Response(200,Dict("Content-Length"=>"15"),"putting user #1")

### Error cases

Invalid HTTP method results in a `405`

In [5]:
router(Request(IOBuffer("DELETE /user/1")))

Response(405,Dict("Allow"=>"GET, PUT"),nothing)

Unless it was an unhandled OPTIONS request. In which case it will be a `204`

In [6]:
router(Request(IOBuffer("OPTIONS /user/1")))

Response(204,Dict("Allow"=>"GET, PUT"),nothing)

An incorrect path will result in a `404` response and it will include a somewhat helpful message in the body

In [7]:
router(Request(IOBuffer("GET /user")))

Response(405,Dict("Allow"=>""),nothing)

In [8]:
router(Request(IOBuffer("GET /users")))

Response(404,Dict("Content-Length"=>"12"),"invalid path")